In [ ]:
%load_ext autoreload
%autoreload 2
%aimport -raw_data_preprocessing -pandas -numpy

In [ ]:
import sys

import pandas as pd
import numpy as np

sys.path.append("../")
sys.path.append("../ukbb_preprocessing/")

from raw_data_preprocessing.raw_data_loader import raw_data_loader
from raw_data_preprocessing.constants import *
from utils import rename_variables, DataRegisterer, percentile, save_results

loader = raw_data_loader()
registerer = DataRegisterer()

data_asset = loader.ws.data.get(name="clin_ascvd", version="6")

In [ ]:
df = pd.read_csv(data_asset.path)
prs_df = rename_variables(df)
df.set_index('IID', inplace=True)
cols = prs_df.columns.str.startswith('eprs') | prs_df.columns.str.startswith('prs') | prs_df.columns.str.startswith('ascvd_')
prs_df = prs_df.loc[:, cols]

# QC

In [ ]:
# compute auc
from sklearn.metrics import roc_auc_score

for disease in ["cad", "cvd", "ht", "iss"]:
    print(disease.upper())
    for prs in ["prs", "eprs"]:

        y_true = df.ascvd_10yr_label.values
        y_score = prs_df[f"{prs}_{disease}"].values

        mask = ~np.isnan(y_true) & ~np.isnan(y_score)
        y_true = y_true[mask]
        y_score = y_score[mask]

        print(f"\t{prs.upper()}:\t{roc_auc_score(y_true, y_score)}")

# Save

In [ ]:
for disease in ["cad", "cvd", "ht", "iss"]:

    tag = f"prs_{disease}"

    score = prs_df[[tag]].copy()
    score["y_score"] = score[tag]

    threshold = score['y_score'].aggregate(percentile(0.95))

    score["y_pred"] = (score["y_score"] > threshold).astype(int)

    score = score[['y_score', 'y_pred']]
    save_results(score, f"results/{tag}")